In [10]:
import numpy as np
import pandas as pd
import csv

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# load data from csv
print("Load data...")
data = pd.read_csv('dataset.csv', keep_default_na=False)
dataset = data.values[:,1:]

# memory and prices parse to integer
print("Preprocessing data...")
prices = np.ndarray(dataset[:,-1].shape)
mem = np.ndarray(dataset[:,4].shape)
c = 0

for p in dataset[:,-1]:
	prices[c] = int(p.split('$')[1])
	c+=1

dataset[:,-1] = prices
c=0

for m in dataset[:,4]:
	try:
		mem[c] = int(m.split('gb')[0])
	except AttributeError:
		mem[c] = 0
	except ValueError:
		mem[c] = 0
	finally:
		c+=1

dataset[:,4] = mem


# all strings parse to binary arrays that mean presence or not
enc = OneHotEncoder(handle_unknown='ignore',sparse=False)

r = np.reshape(dataset[:,0],(10484,1))

brand=enc.fit_transform(r)
color=enc.fit_transform(np.reshape(dataset[:,1],(10484,1)))
condition=enc.fit_transform(np.reshape(dataset[:,2],(10484,1)))
contract=enc.fit_transform(np.reshape(dataset[:,3],(10484,1)))
mobOs=enc.fit_transform(np.reshape(dataset[:,5],(10484,1)))
model=enc.fit_transform(np.reshape(dataset[:,6],(10484,1)))

mem = mem.reshape((10484,1))
prices = prices.reshape((10484,1))

# adding data
print("Creating data partitions...")
all_data = np.concatenate((brand,color,condition,contract,mobOs,model,mem,prices),axis=1)

# split data set on training data (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(all_data[:,0:-1], all_data[:,-1], test_size=0.2, shuffle=False)

# creating model KNN
print("Training model...")
knnR = KNeighborsRegressor(n_neighbors=5, metric='euclidean', weights='uniform', leaf_size=30)

# fit model on training data
knnR.fit(X_train, y_train)

# scores on R square
print("Results:")
print("\taccuracy training: " + str(knnR.score(X_train, y_train)))
print("\taccuracy test: " + str(knnR.score(X_test, y_test)))

y_hat = knnR.predict(X_test)
y_nn = knnR.kneighbors(X_test, return_distance=False)


Load data...
Preprocessing data...
Creating data partitions...
Training model...
Results:
	accuracy training: 0.9382211367676482
	accuracy test: 0.8862634990119136


In [11]:
y_nn = knnR.kneighbors(X_test[0].reshape(1,-1), return_distance=False)
X_test[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [14]:
y_hat = knnR.predict(X_test[2].reshape(1,-1))

y_hat

array([318.])